In [ ]:
from scipy.stats import ttest_1samp
import numpy as np
import pandas as pd


from scipy.stats import norm, t, kstest, shapiro
import statsmodels.api as sm
from matplotlib import pyplot as plt


import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [ ]:
np.random.seed(17)

In [ ]:
# Создадим выборку нормального распределения и распределения Стьюдента с 5-ю степенями свободы.
# По форме они очень похожи..
x = norm.rvs(size = 250)
y = t.rvs(size = 250, df = 5)

In [ ]:
import plotly.express as px # Интерактивная библиотека для графиков.
# marginal = 'box' - указываем, чтобы вывел еще и boxplot

fig = px.histogram(y, x=y, title='y', marginal = 'box')
fig.show(renderer='colab')

In [ ]:
# Смотрим и для второго распределения x.
# Выглядит нормальным, но является ли нормальным - нужно выяснить.


fig = px.histogram(x, x=x, title='x', marginal = 'box')
fig.show(renderer='colab')

Проверка на нормальность:
1) Графические: Оценка данных с использованием гистограммы и Quantile-Quantile-plot

2) Статистические: Критерии проверки нормальности распределения (Шапиро-Уилка при n < 5_000 и Колмогорова-Смирнова для любых выборок)

In [ ]:
# QQ-plot для x и для y

fig = sm.qqplot(y, color='green')
plt.title('Normal Q-Q Plot')
plt.show

In [ ]:
fig = sm.qqplot(x, color='green')
plt.title('Normal Q-Q Plot')
plt.show

Тест Колмогорова-Смирнова - проверка на нормальность.
Для выборок любого размера.

H0  - x взята из генеральной совокупности с нормальным распределением.

H1  - x взята из ген.совокупности с другим распределением.

Можно использовать для проверки любого другого распределения(Стьюдента, биномиального, коши, экспоненциального).

При p-value > alpha: анализируемое распределение не отличается от нормального.

In [ ]:
# Проверим принадлежность выборок к нормальному распределению.

print(kstest(x, 'norm'))
print(kstest(y, 'norm'))

По этим выборкам данные распределения не отличаются от нормального.

In [ ]:
# Проверим принадлежность к экспоненциальному распределению (сложная гипотеза):

print(kstest(x, 'expon'))

In [ ]:
# Проверим принадлежность к биномиальному распределению (простая гипотеза)
# Требует уточнения параметров.

print(kstest(x, 'binom', args=(1_500, 0.16)))

Наше распределение точно отличается от экспоненциального и от биномиального

Проверка на нормальность Шапиро-Уилка:
Является наиболее мощным для воборок до 5_000 наблюдений.

H0  - x взята из генеральной совокупности с нормальным распределением.

H1  - x взята из ген.совокупности с другим распределением.

При p-value > alpha: анализируемое распределение не отличается от нормального.

In [ ]:
w, p_val = shapiro(x)
print('Shapiro-Wilk normality test for x')
print('---' * 10)
print('w =', w)
print('p-value =', p_val)

Распределение x не отличается от нормального.

p-value > alpha

In [ ]:
w, p_val = shapiro(y)
print('Shapiro-Wilk normality test for x')
print('---' * 10)
print('w =', w)
print('p-value =', p_val)

Тест Колмагорова-Смирнова показал, что оба распределения нормальные.

НО!!! тест Шапиро-Уилка является более мощным на выборках до 5_000 => он и выявил, то распределение y отличается от нормального. (Мы как раз и получили его как распределение t-Стьюдента)

Расчет мощности теста / расчет длительности эксперимента.

Факторы, влияющие на мощность теста:
1) статистические:

Размер выбора(effect_size, ошибка 1 рода, ошибка 2 рода)
Дисперсия (дисперсия частично зашита в effect_size => чем больше дисперсия, тем больше нам нужно наблюдений)
2) продуктовые:

Сезонность(недельная/месячная...)
Метрика (тип метрики, окно закрытия метрики)

In [ ]:
import math
import numpy as np
import statsmodels.stats.power as smp
from tqdm.auto import tqdm  # будет показывать количество итераций
import matplotlib.pyplot as plt
import seaborn as sns       # библиотека для визуализаций

In [ ]:
plt.style.use('ggplot')

In [ ]:
# Критерий пропорций (для кликов, конверсий)

alpha = 0.05
power = 0.9
n = 450             # Количество наблюдений.
p_x = 0.5           # Конверсии.
p_y = 0.6

In [ ]:
# effect_size

h = 2 * math.asin(np.sqrt(p_x)) - 2 * math.asin(np.sqrt(p_y))
h

Test Procedure
If we assume that  P1  and  P2  represents the two proportions. The effect_size is represented as this difference:

h=φ1−φ2 , where  φ=arcsine(Pi−−√)

In [ ]:
# Мощность критерия для выборки из n элементов, если effect_size = h, alpha = 5%

power = smp.zt_ind_solve_power(effect_size=h, nobs1=n, alpha=alpha, alternative='two-sided')
power

In [ ]:
# Количество наблюдений, необходимое для заданного эффекта при alpha=5%, beta(power)=80%

smp.zt_ind_solve_power(effect_size=h, alpha=alpha, power=power, alternative='two-sided')

In [ ]:
# Величина эффекта effect_size при заданных alpha и beta

smp.zt_ind_solve_power(nobs1=n, alpha=alpha, power=power)

In [ ]:
effects = []
sample_sizes = []

for i in tqdm(range(10, 10_000)):
  effects.append(smp.tt_ind_solve_power(nobs1=i, alpha=alpha, power=power))
  sample_sizes.append(i)

In [ ]:
df = pd.DataFrame({'effects': effects, 'sample_sizes': sample_sizes})
df

In [ ]:
import plotly.express as px

fig = px.line(df, x='sample_sizes', y='effects', title='effect vs sample size')
fig.show(renderer='colab')

Чем меньше эффект, тем больше нам нужно количество наблюдений, чтобы его зафиксировать.

In [ ]:
power = []
sample_sizes = []

for i in tqdm(range(10, 10_000)):
  power.append(smp.tt_ind_solve_power(nobs1=i, alpha=alpha, effect_size=0.1))
  sample_sizes.append(i)

In [ ]:
df2 = pd.DataFrame({'power': power, 'sample_sizes': sample_sizes })
df2

In [ ]:
fig = px.line(df2, x='sample_sizes', y='power', title='power vs sample size')
fig.show(renderer='colab')

При росте количества наблюдений, мощность увеличивается.

Множественная проверка гипотез

In [ ]:
import numpy as np
import matplotlib.pyplot as  plt
from scipy import stats

rvs_1 = stats.norm.rvs(loc=5, scale=10, size=1_000, random_state=0)
rvs_2 = stats.norm.rvs(loc=6.5, scale=8, size=1_000, random_state=0)

In [ ]:
def t_test_function(rvs, alpha, no_test):   # на вход: распределение, альфу и количество тестов.
  counter=0
  for i in range(no_test):
    rvs_random = stats.norm.rvs(loc=5, scale=10, size=1_000, random_state=i+1)

    statistic, pvalue = stats.ttest_ind(rvs, rvs_random, equal_var=False)

    if pvalue <= alpha:
      counter += 1

  print(counter)

# поправка Бонферони: альфа / на количество попарных сравнений.

In [ ]:
def bonferroni_correction_function(rvs, alpha, no_test):
  alpha_bonferroni = alpha / no_test

  counter = 0
  for i in range(no_test):
    rvs_random = stats.norm.rvs(loc=5, scale=10, size=1_000, random_state=i+1)

    statistic, pvalue = stats.ttest_ind(rvs, rvs_random, equal_var=False)

    if pvalue <= alpha_bonferroni:
      counter += 1

  print(counter)

In [ ]:
t_test_function(rvs_1, alpha=0.05, no_test=100)
t_test_function(rvs_2, alpha=0.05, no_test=100)

bonferroni_correction_function(rvs_1, alpha=0.05, no_test=100)

bonferroni_correction_function(rvs_2, alpha=0.05, no_test=100)

В 0 раз - отличается статистически значимая разница. Поправка Бонферони понижает риск False Positive.

Текст, выделенный полужирным шрифтом# Bootstrap.

In [ ]:
!pip install bootstrapped

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

import bootstrapped.bootstrap as bs
import bootstrapped.compare_functions as bs_compare
import bootstrapped.stats_functions as bs_stats


In [ ]:
plt.style.use('ggplot')

In [ ]:
n = 11_000
sample_A = np.random.exponential(scale=1/0.002, size=n)
sample_B = np.random.exponential(scale=1/0.00201, size=n)

df = pd.DataFrame({"sample_A": sample_A, "sample_B": sample_B})

In [ ]:
s_1 = df.sample_A.values

s_2 = df.sample_B.values

b = bs.bootstrap_ab(s_1, s_2, stat_func=bs_stats.mean,
                    compare_func=bs_compare.difference, alpha=0.05, num_iterations=10_000)

print(b.lower_bound, b.upper_bound)

In [ ]:
# Для стандартного отклонения

v = bs.bootstrap_ab(s_1, s_2, stat_func=bs_stats.std, 
                    compare_func=bs_compare.difference, alpha=0.05, num_iterations=10_000)

print(v.lower_bound, v.upper_bound)

Бакетирование.
разбивает данные на более управляемые части (сегменты или бакеты), чтобы ускорить последовательные чтения данных для последующих заданий. В один бакет попадают строчки таблицы, у которых совпадает значение хэш-функции, вычисленное по определенной колонке.

Способ привести распределение к нормальному и применить, например, ttest.

Когда чего применяем:
сохранить информацию о дисперсии и среднем в выборке до трансформации.
привести к нормальному распределению.
Возьмем кратное количество групп: например, 2_000 (можно 300 или 600)

In [ ]:
plt.style.use('ggplot')

In [ ]:
b = 2_000
n = 200_000

val_A = np.random.exponential(scale=1/0.002, size=n)
val_B = np.random.exponential(scale=1/0.00201, size=n) 

sample_exp = pd.DataFrame({
    "metric": np.concatenate([val_A, val_B]),
    "group": ["A" for i in range(n)] + ["B" for i in range(n)],
    "bucket": [i for i in range(b)] * int(n * 2 / b)
})

In [ ]:
# Агрегация по бакетам и считаем среднее.

backeted_df = sample_exp.groupby(by=["bucket", "group"])["metric"].agg(mu=np.mean, sd_mu=np.std).reset_index()
backeted_df.head()

In [ ]:
# Сравним выборочное среднее и среднее по баккетам изначальное.

round(np.mean(sample_exp["metric"]), 3) == round(np.mean(backeted_df["mu"]), 3)

In [ ]:
np.var(sample_exp["metric"]) / len(sample_exp["metric"])

In [ ]:
np.var(backeted_df["mu"]) / len(backeted_df["mu"])

In [ ]:
sample_exp

In [ ]:
import plotly.express as px

fig = px.histogram(sample_exp, x=sample_exp.metric, title='x', marginal='box')
fig.show(renderer="colab")